#13. Advanced Graphs (11 Questions)

###***1. Path with Minimum Effort (Medium)***
https://leetcode.com/problems/path-with-minimum-effort/

In [ ]:
from typing import List

class Solution:
  def minimumEffortPath(self, heights: List[List[int]]) -> int:
    ROWS, COLS = len(heights), len(heights[0])

    minHeap = [[0, 0, 0]] # [diff, r, c], max absolute difference
    visit = set()
    directions = [[0, 1], [0, -1], [1, 0], [-1, 0]]

    while minHeap:
      diff, r, c = heapq.heappop(minHeap)

      if (r, c) in visit:
        continue
      visit.add((r, c))
      if (r, c) == (ROWS - 1, COLS - 1): # We will reach to this point any how
        return diff # Result

      for dr, dc in directions:
        newR, newC = r + dr, c + dc
        if (newR < 0 or newC < 0 or newR == ROWS or newC == COLS or (newR, newC) in visit):
          continue
        newDiff = max(diff, abs(heights[r][c] - heights[newR][newC])) # We want max diff
        heapq.heappush(minHeap, [newDiff, newR, newC])

###***2. Reconstruct Itinerary (Hard)***
https://leetcode.com/problems/reconstruct-itinerary/

In [ ]:
from typing import List

class Solution:
  def findItinerary(self, tickets: List[List[str]]) -> List[str]:
    adj = {src:[] for src, dst in tickets}

    tickets.sort() # want to return in smallest lexical order
    for src, dst in tickets: # adjacency list source to destinations
      adj[src].append(dst)

    res = ["JFK"] # as given
    def dfs(src):
      if len(res) == len(tickets) + 1: # visited all dst + starting city
        return True
      if src not in adj: # no outgoing
        return False

      temp = list(adj[src])
      for i, v in enumerate(temp): # index, node
        adj[src].pop(i)
        res.append(v)
        if dfs(v):
          return True
        # Backtracking - when False is returned, no outgoing
        adj[src].insert(i, v)
        res.pop()
      return False

    dfs("JFK")
    return res

In [ ]:
class Solution:
    def findItinerary(self, tickets: List[List[str]]) -> List[str]:
      adj = {src: [] for src, dst in tickets}
      res = []

      for src, dst in tickets:
        adj[src].append(dst)

      for key in adj:
        adj[key].sort()

      def dfs(adj, result, src):
        if src in adj:
          destinations = adj[src][:]
          while destinations:
            dest = destinations[0]
            adj[src].pop(0)
            dfs(adj, res, dest)
            destinations = adj[src][:]
        res.append(src)

      dfs(adj, res, "JFK")
      res.reverse()

      if len(res) != len(tickets) + 1:
        return []

      return res

###***3. Min Cost to Connect All Points (Medium)***
https://leetcode.com/problems/min-cost-to-connect-all-points/

In [ ]:
from typing import List

class Solution:
  def minCostConnectPoints(self, points: List[List[int]]) -> int:
    # Building adjacency list to use in Prim's algorithm
    N = len(points)
    adj = {i: [] for i in range(N)}  # i : list of [cost, node]
    for i in range(N):
      x1, y1 = points[i]
      for j in range(i + 1, N):
        x2, y2 = points[j]
        dist = abs(x1 - x2) + abs(y1 - y2)
        adj[i].append([dist, j])
        adj[j].append([dist, i])

    # Prim's
    res = 0
    visit = set()
    minH = [[0, 0]]  # [cost, point]
    while len(visit) < N:
      cost, i = heapq.heappop(minH)
      if i in visit:
        continue
      res += cost
      visit.add(i)
      for neiCost, nei in adj[i]:
        if nei not in visit:
          heapq.heappush(minH, [neiCost, nei])
    return res

# Prim's Minimum Spanning Tree

###***4. Network Delay Time (Medium)***
https://leetcode.com/problems/network-delay-time/

In [ ]:
from typing import List

class Solution:
  def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:
    edges = collections.defaultdict(list)
    for u, v, w in times:
      edges[u].append((v, w))

    minHeap = [(0, k)] # weight, node - k is the starting node
    visit = set()
    t = 0 # weight
    while minHeap:
      w1, n1 = heapq.heappop(minHeap)
      if n1 in visit: # shortest dist node will be in the visit set first
        continue
      visit.add(n1)
      t = max(t, w1)

      for n2, w2 in edges[n1]: # visiting neighbors
        if n2 not in visit:
          heapq.heappush(minHeap, (w1 + w2, n2)) # w1 + w2 is total weight to reach n2

    return t if len(visit) == n else -1

# Dijkstra Algorithm

###***5. Path with Maximum Probability (Medium)***
https://leetcode.com/problems/path-with-maximum-probability/

In [ ]:
from typing import List

class Solution:
  def maxProbability(self, n: int, edges: List[List[int]], succProb: List[float], start: int, end: int) -> float:
    adj = collections.defaultdict(list)
    for i in range(len(edges)):
      src, dst = edges[i]
      adj[src].append([dst, succProb[i]])
      adj[dst].append([src, succProb[i]])

    # Priority queue, maxHeap
    pq = [(-1, start)] # (prob, node)
    visit = set()

    while pq:
      prob, cur = heapq.heappop(pq)
      visit.add(cur)

      if cur == end: # If we find the soultion, it is gonna be returned here
        return prob * -1 # Changing negative val to positive

      for nei, edgeProb in adj[cur]:
        if nei not in visit:
          heapq.heappush(pq, (prob * edgeProb, nei))

    return 0 # Didn't find a solution

###***6. Swim In Rising Water (Hard)***
https://leetcode.com/problems/swim-in-rising-water/

In [ ]:
from typing import List

class Solution:
  def swimInWater(self, grid: List[List[int]]) -> int:
    N = len(grid)
    visit = set()
    minH = [[grid[0][0], 0, 0]] # (time or max_height, r, c)
    directions = [[0, 1], [0, -1], [1, 0], [-1, 0]]

    visit.add((0, 0))
    while minH:
      t, r, c, = heapq.heappop(minH)
      if r == N - 1 and c == N - 1: # reached the bottom right or dst
        return t
      for dr, dc in directions:
        neiR, neiC = r + dr, c + dc
        if (neiR < 0 or neiC < 0 or
            neiR == N or neiC == N or
            (neiR, neiC) in visit):
          continue
        visit.add((neiR, neiC))
        heapq.heappush(minH, [max(t, grid[neiR][neiC]), neiR, neiC])

###***7. Alien Dictionary (Hard)***
https://leetcode.com/problems/alien-dictionary/

https://www.lintcode.com/problem/892/

In [ ]:
# Topological Sort
from typing import List

class Solution:
  def alienOrder(self, words: List[str]) -> str:
    adj = {c:set() for w in words for c in w}

    for i in range(len(words) - 1):
      w1, w2 = words[i], words[i + 1]
      minLen = min(len(w1), len(w2))
      if len(w1) > len(w2) and w1[:minLen] == w2[:minLen]:
        return ""
      for j in range(minLen):
        if w1[j] != w2[j]:
          adj[w1[j]].add(w2[j])
          break

      visit = {} # False=visited, True=current path
      res = []

      def dfs(c):
        if c in visit:
          return visit[c]

        visit[c] = True
        for nei in adj[c]:
          if dfs(nei):
            return True
        visit[c] = False
        res.append(c)

      for c in adj:
        if dfs(c):
          return ""

      res.reverse()
      return "".join(res)

###***8. Cheapest Flights Within K Stops (Medium)***
https://leetcode.com/problems/cheapest-flights-within-k-stops/

In [ ]:
from typing import List

class Solution:
  def findCheapestPrice(self, n: int, flights: List[List[int]], src: int, dst: int, k: int) -> int:
    prices = [float("inf")] * n
    prices[src] = 0

    for i in range(k + 1): # need to run loop k + 1 times
      tmpPrices = prices.copy()

      for s, d, p in flights: # a=source, d=destination, p=price
        if prices[s] == float("inf"):
          continue
        if prices[s] + p < tmpPrices[d]:
          tmpPrices[d] = prices[s] + p
      prices = tmpPrices

    return -1 if prices[dst] == float("inf") else prices[dst]

# Bellman-Ford

###***9. Number of Good Paths (Hard)***
https://leetcode.com/problems/number-of-good-paths/

In [3]:
from typing import List

class UnionFind:
  def __init__(self, n):
    self.par = list(range(n))
    self.rank = [0] * n

  def find(self, i):
    while i != self.par[i]:
      self.par[i] = self.par[self.par[i]] # Path compression
      i = self.par[i]
    return i

  def union(self, a, b):
    aRoot = self.find(a)
    bRoot = self.find(b)

    if aRoot == bRoot:
      return False
    if self.rank[aRoot] < self.rank[bRoot]:
      self.par[aRoot] = bRoot
      self.rank[bRoot] += self.rank[aRoot]
    else:
      self.par[bRoot] = aRoot
      self.rank[aRoot] += self.rank[bRoot]
    return True

class Solution:
  def numberOfGoodPaths(self, vals: List[int], edges: List[List[int]]) -> int:
    adj = collections.defaultdict(list)
    for a, b in edges:
      adj[a].append(b)
      adj[b].append(a)

    valToIndex = collections.defaultdict(list)
    for i, val in enumerate(vals):
      valToIndex[val].append(i)

    res = 0
    uf = UnionFind(len(vals))
    for val in sorted(valToIndex.keys()):
      for i in valToIndex[val]:
        for nei in adj[i]:
          if vals[nei] <= vals[i]:
            uf.union(nei, i)

      # For each disjoint set, how many val's does it contain?
      count = collections.defaultdict(int)
      for i in valToIndex[val]:
        count[uf.find(i)] += 1
        res += count[uf.find(i)]
    return res

###***10. Remove Max Number of Edges to Keep Graph Fully Traversable (Hard)***
https://leetcode.com/problems/remove-max-number-of-edges-to-keep-graph-fully-traversable/

In [2]:
from typing import List

class UF: # Union Find/Disjoint Set
  def __init__(self, n):
    self.n = n
    self.par = [i for i in range(n + 1)]
    self.rank = [1] * (n + 1)

  def find(self, x):
    while x != self.par[x]:
      self.par[x] = self.par[self.par[x]]
      x = self.par[x]
    return x

  # Return 1 if success, 0 otherwise
  def union(self, x1, x2):
    p1, p2 = self.find(x1), self.find(x2)
    if p1 == p2:
      return 0
    if self.rank[p1] > self.rank[p2]:
      self.rank[p1] += self.rank[p2]
      self.par[p2] = p1
    else:
      self.rank[p2] += self.rank[p1]
      self.par[p1] = p2
    self.n -= 1
    return 1

  def isConnected(self):
    return self.n <= 1

class Solution:
  def maxNumEdgesToRemove(self, n: int, edges: List[List[int]]) -> int:
    alice, bob = UF(n), UF(n)
    cnt = 0 # Num of edges we have to keep

    for t, src, dst in edges:
      if t == 3:
        cnt += (alice.union(src, dst) | bob.union(src, dst)) # Basically increasing the count by 1

    for t, src, dst in edges:
      if t == 1:
        cnt += alice.union(src, dst)
      elif t == 2:
        cnt += bob.union(src, dst)

    if bob.isConnected() and alice.isConnected():
      return len(edges) - cnt # Total edges - Total count to keep fully traversable

    return -1

###***11. Find Critical and Pseudo Critical Edges in Minimum Spanning Tree (Hard)***
https://leetcode.com/problems/find-critical-and-pseudo-critical-edges-in-minimum-spanning-tree/

In [4]:
from typing import List

class UnionFind: # Union Find
  def __init__(self, n):
    self.par = [i for i in range(n)]
    self.rank = [1] * n

  def find(self, v1):
    while v1 != self.par[v1]:
      self.par[v1] = self.par[self.par[v1]]
      v1 = self.par[v1]
    return v1

  def union(self, v1, v2):
    p1, p2 = self.find(v1), self.find(v2)
    if p1 == p2:
      return False
    if self.rank[p1] > self.rank[p2]:
      self.par[p2] = p1
      self.rank[p1] += self.rank[p2]
    else:
      self.par[p1] = p2
      self.rank[p2] += self.rank[p1]
    return True

class Solution:
  def findCriticalAndPseudoCriticalEdges(self, n: int, edges: List[List[int]]) -> List[List[int]]:
    # Time: O(E^2) - UF operations are assumed to be approx O(1)
    for i, e in enumerate(edges):
      e.append(i) # [v1, v2, weight, original_index]

    edges.sort(key=lambda e: e[2]) # Sorting by weight

    mst_weight = 0
    uf = UnionFind(n)
    for v1, v2, w, i in edges:
      if uf.union(v1, v2):
        mst_weight += w

    critical, pseudo = [], []
    for n1, n2, e_weight, i in edges:
      # Try without curr edge
      weight = 0
      uf = UnionFind(n)
      for v1, v2, w, j in edges:
        if i != j and uf.union(v1, v2): # Union is successful
          weight += w
      if max(uf.rank) != n or weight > mst_weight:
        critical.append(i)
        continue

      # Try with curr edge
      uf = UnionFind(n)
      uf.union(n1, n2)
      weight = e_weight
      for v1, v2, w, j in edges:
        if uf.union(v1, v2):
          weight += w
      if weight == mst_weight:
        pseudo.append(i)
    return [critical, pseudo]